# Importando dados e bibliotecas

In [1]:
library(tidyverse)
library(arrow)
library(caret)
library(ggthemes)
set.seed(123)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.0     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.1     ✔ tibble    3.2.0
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp


Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




In [2]:
dados <- read_parquet('../Dados/Processado/Extinguisher_Fire.parquet')
head(dados)

SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,gasoline,10,96,0.0,75,0
1,gasoline,10,96,0.0,72,1
1,gasoline,10,96,2.6,70,1
1,gasoline,10,96,3.2,68,1
1,gasoline,10,109,4.5,67,1
1,gasoline,10,109,7.8,66,1


# Treinamento do Modelo

Durante a análise exploratória de dados, foi chegado a conclusão de que não haveria tanta necessidade de processar os dados antes de aplicar os modelos de machine learning. A classe que queremos prever já está balanceada tanto em termos de quantidade total quanto por tipo de combustível. Além disso, não há dados faltantes. O único problema que precisa ser tratado é a diferença de escala dos dados. Dessa forma, foi criado algumas funções para aplicar normalização dos dados, treinamento de modelos genéricos e avaliação dos modelos. 

Primeiramente, é necessário entender quais os algoritmos que funcionam melhor com o modelo de dados que temos. Para isso, será treinado diversos modelos diferentes 

In [3]:
split_data <- function(data, split) {
  index <- createDataPartition(data$STATUS, p = split, list = FALSE)
  train <- data[index, ]
  test <- data[-index, ]
  return(list(train, test))
}

train_evaluate_model <- function(model, data, split) {
  data_list <- split_data(data, split)

  model_trained <- train(
    STATUS ~ .,
    data = data_list[[1]],
    preProcess = c("center", "scale"),
    method = model,
    trControl = trainControl(method = "cv", number = 10)
  )

  confusion_matrix <- confusionMatrix(predict(model_trained, data_list[[2]]), data_list[[2]]$STATUS)
  accuracy <- confusion_matrix$overall['Accuracy']
  kappa <- confusion_matrix$overall['Kappa']
  precision <- confusion_matrix$byClass['Precision']
  recall <- confusion_matrix$byClass['Recall']
  f1 <- confusion_matrix$byClass['F1']
  auc <- confusion_matrix$byClass['Balanced Accuracy']

  eval_df <- data.frame(
    model,
    accuracy,
    kappa,
    precision,
    recall,
    f1,
    auc
  )

  rownames(eval_df) <- NULL

  return(list(model_trained, eval_df, confusion_matrix))
}

train_multiple_models <- function(models_vector, data, split) {
  models <- list()
  for(model in models_vector) {
    message("Iniciando treinamento do modelo ", model, "...")
    models[[model]] <- train_evaluate_model(model, data, split)
    message("Treinamento do modelo ", model, " finalizado !!!")
  }

  return(models)
}

create_evaluate_df <- function(models_list) {
  evaluate_df <- data.frame()
  for(model in names(models_list)) {
    evaluate_df <- rbind(evaluate_df, models_list[[model]][[2]])
  }

  return(evaluate_df)
}

In [4]:
models_list <- train_multiple_models(c("glm", "rpart", "rf", "svmRadial", "xgbTree"), dados, 0.7)
evaluate_df <- create_evaluate_df(models_list)

Iniciando treinamento do modelo glm...

Treinamento do modelo glm finalizado !!!

Iniciando treinamento do modelo rpart...

Treinamento do modelo rpart finalizado !!!

Iniciando treinamento do modelo rf...

Treinamento do modelo rf finalizado !!!

Iniciando treinamento do modelo svmRadial...

Treinamento do modelo svmRadial finalizado !!!

Iniciando treinamento do modelo xgbTree...



[19:01:39] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[19:01:39] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[19:01:39] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[19:01:39] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[19:01:39] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[19:01:39] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[19:01:39] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[19:01:39] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[19:01:39] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is deprecated, use `iteration_range` instead.
[19:01:39] WARNING: src/c_api/c_api.cc:935: `ntree_limit` is dep

Treinamento do modelo xgbTree finalizado !!!



# Avaliação dos múltiplos modelos

Como podemos observar na tabela abaixo, teve dois modelos que se destacaram em diante os demais. O primeiro foi o modelo de Random Forest o segundo foi o XGBoost. Ambas as métricas de avaliação dos modelos foram similares, assim foi necessário buscar outras maneiras de avaliar os modelos. A maneira utilizada foi a matriz de confusão. A matriz de confusão é uma tabela que mostra a quantidade de acertos e erros de cada modelo. No caso do modelo de Random Forest, o modelo teve 84 falsos negativos e 83 falsos positivos. Já no modelo de XGBoost, o modelo teve 97 falsos negativos e 74 falsos positivos. Dado o problema em questão, isto é, prever a efiência de um extindor de incêndio, é melhor ter um modelo que tenha mais falsos positivos do que falsos negativos. Isso porque, se o modelo prever que o extintor não apagará o fogo, mas apagar, o prejuízo será menor do que se o modelo prever que o extintor apagará o fogo, mas não apagar. Dessa forma, o modelo de XGBoost foi escolhido para ser utilizado no modelo final.

In [5]:
evaluate_df %>%
    arrange(desc(f1), desc(recall))

model,accuracy,kappa,precision,recall,f1,auc
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
rf,0.9680749,0.9361485,0.9680365,0.9684050,0.9682207,0.9680735
xgbTree,0.9673103,0.9346167,0.9633962,0.9718310,0.9675952,0.9672903
svmRadial,0.9462818,0.8925573,0.9426415,0.9508946,0.9467500,0.9462614
glm,0.9004015,0.8007713,0.8885609,0.9166349,0.9023796,0.9003298
rpart,0.8864462,0.7729277,0.9028934,0.8671488,0.8846602,0.8865314


In [6]:
models_list$rf[[3]]$table

          Reference
Prediction    0    1
         0 2544   84
         1   83 2520

In [7]:
models_list$xgbTree[[3]]$table

          Reference
Prediction    0    1
         0 2553   97
         1   74 2507

# Tunagem do Modelo Final

Agora que sabemos qual o melhor algoritmo a ser seguido, vamos testar alguns parâmetros para ver se conseguimos melhorar ainda mais o resultado. A variável tune_grid_v1 refere-se aos parametros que o modelo será treinado e testado. A conclusão desse teste é de que foi possível melhorar o resultado do modelo, em sua melhor versão, o modelo conseguiu reduzir drasticamente a quantidade de falsos positivo em cerca de 25%. No problema em questão, isso implica em diminuir a quantidade de extintores que serão trocados por não serem capazes de apagarem o fogo, mas que na verdade era sim capazes.

In [8]:
train_xgboost <- function(data, train_control, tune_grid) {
  data_list <- split_data(data, 0.7)

  model_trained <- train(
    STATUS ~ .,
    data = data_list[[1]],
    preProcess = c("center", "scale"),
    method = "xgbTree",
    trControl = train_control,
    tuneGrid = tune_grid,
    verbosity = 0
  )

  confusion_matrix <- confusionMatrix(predict(model_trained, data_list[[2]]), data_list[[2]]$STATUS)
  accuracy <- confusion_matrix$overall['Accuracy']
  kappa <- confusion_matrix$overall['Kappa']
  precision <- confusion_matrix$byClass['Precision']
  recall <- confusion_matrix$byClass['Recall']
  f1 <- confusion_matrix$byClass['F1']
  auc <- confusion_matrix$byClass['Balanced Accuracy']

  eval_df <- data.frame(
    accuracy,
    kappa,
    precision,
    recall,
    f1,
    auc
  )

  rownames(eval_df) <- NULL

  return(list(model_trained, eval_df, confusion_matrix))
}

In [9]:
train_control_v1 <- trainControl(
  method = "cv",
  number = 10
)

tune_grid_v1 <- expand.grid(
  nrounds = 100,
  max_depth = c(3, 5),
  eta = c(0.1, 0.3),
  gamma = c(0, 3, 6),
  colsample_bytree = c(0.5, 0.75, 1),
  min_child_weight = 1,
  subsample = c(0.5, 0.75, 1)
)

models_list_v1 <- train_xgboost(dados, train_control_v1, tune_grid_v1)

In [13]:
print('Modelo novo:')

models_list_v1[[3]]$table

print('Modelo antigo:')
models_list[[5]][[3]]$table

[1] "Modelo novo:"


          Reference
Prediction    0    1
         0 2555   74
         1   72 2530

[1] "Modelo antigo:"


          Reference
Prediction    0    1
         0 2553   97
         1   74 2507

In [17]:
models_list_v1[[1]]$finalModel

##### xgb.Booster
raw: 225.2 Kb 
call:
  xgboost::xgb.train(params = list(eta = param$eta, max_depth = param$max_depth, 
    gamma = param$gamma, colsample_bytree = param$colsample_bytree, 
    min_child_weight = param$min_child_weight, subsample = param$subsample), 
    data = x, nrounds = param$nrounds, objective = "binary:logistic", 
    verbosity = 0)
params (as set within xgb.train):
  eta = "0.3", max_depth = "5", gamma = "0", colsample_bytree = "1", min_child_weight = "1", subsample = "1", objective = "binary:logistic", verbosity = "0", validate_parameters = "TRUE"
xgb.attributes:
  niter
callbacks:
  cb.print.evaluation(period = print_every_n)
# of features: 8 
niter: 100
nfeatures : 8 
xNames : SIZE FUELkerosene FUELlpg FUELthinner DISTANCE DESIBEL AIRFLOW FREQUENCY 
problemType : Classification 
tuneValue :
	   nrounds max_depth eta gamma colsample_bytree min_child_weight subsample
90     100         5 0.3     0                1                1         1
obsLevels : 0 1 
par